In [1]:
import polars as pl 
import pandas as pd
# import sys
from time import perf_counter

# Data Loading 

In [2]:
start = perf_counter()
df_pl = pl.read_csv("./data/data1.csv", skip_rows=2) # Not lazy read.
print(len(df_pl))
print(df_pl.columns)
end = perf_counter()
print(f"Spent {round(end-start,2)}s.")

567898
['Day', 'Region', 'Metric1', 'Metric2', 'Metric3']
Spent 0.03s.


In [3]:
start = perf_counter()
df_pd = pd.read_csv("./data/data1.csv", skiprows=2)
print(len(df_pd))
print(df_pd.columns)
end = perf_counter()
print(f"Spent {round(end-start,2)}s.")

567898
Index(['Day', 'Region', 'Metric1', 'Metric2', 'Metric3'], dtype='object')
Spent 0.23s.


# Clean Zip Code

1. Make zips like 1234 into 01234
2. Get rid of everything after '-'. E.g. 01234-123 ---> 01234

In [4]:
df1_pl = pl.read_csv("./data/data2.csv")
df1_pd = pd.read_csv("./data/data2.csv")

In [5]:
df1_pl.limit(5)

day,country,postal_code,total_sales,orders
str,str,str,f64,i64
"""7/1/2022""","""United States""","""84115""",6.384645,1
"""7/1/2022""","""United States""","""91326-1749""",2.11396,1
"""7/1/2022""","""United States""","""93103""",20.224295,1
"""7/1/2022""","""United States""","""2465""",94.73113,1
"""7/1/2022""","""United States""","""80108""",52.652385,1


In [6]:
print(df1_pd.head())
print(df1_pd.dtypes)

        day        country postal_code  total_sales  orders
0  7/1/2022  United States       84115     6.384645       1
1  7/1/2022  United States  91326-1749     2.113960       1
2  7/1/2022  United States       93103    20.224295       1
3  7/1/2022  United States        2465    94.731130       1
4  7/1/2022  United States       80108    52.652385       1
day             object
country         object
postal_code     object
total_sales    float64
orders           int64
dtype: object


In [7]:
start = perf_counter()
df1_pl = df1_pl.with_columns(
            pl.col('postal_code').str.splitn("-",2).struct.field("field_0").str.rjust(5, "0").alias('postal_code_2')
        )
print(df1_pl.limit(5))
end = perf_counter()
print(f"Spent {round(end-start,3)}s.")

shape: (5, 6)
┌──────────┬───────────────┬─────────────┬─────────────┬────────┬───────────────┐
│ day      ┆ country       ┆ postal_code ┆ total_sales ┆ orders ┆ postal_code_2 │
│ ---      ┆ ---           ┆ ---         ┆ ---         ┆ ---    ┆ ---           │
│ str      ┆ str           ┆ str         ┆ f64         ┆ i64    ┆ str           │
╞══════════╪═══════════════╪═════════════╪═════════════╪════════╪═══════════════╡
│ 7/1/2022 ┆ United States ┆ 84115       ┆ 6.384645    ┆ 1      ┆ 84115         │
│ 7/1/2022 ┆ United States ┆ 91326-1749  ┆ 2.11396     ┆ 1      ┆ 91326         │
│ 7/1/2022 ┆ United States ┆ 93103       ┆ 20.224295   ┆ 1      ┆ 93103         │
│ 7/1/2022 ┆ United States ┆ 2465        ┆ 94.73113    ┆ 1      ┆ 02465         │
│ 7/1/2022 ┆ United States ┆ 80108       ┆ 52.652385   ┆ 1      ┆ 80108         │
└──────────┴───────────────┴─────────────┴─────────────┴────────┴───────────────┘
Spent 0.008s.


In [11]:
start = perf_counter()
# df1_pd["postal_code"].str.split(pat="-",n=2,expand=False).str[0].str.rjust(5, "0")
# [x.split("-")[0].rjust(5, "0") for x in df1_pd["postal_code"]]
df1_pd["postal_code_2"] = df1_pd["postal_code"].str.split(pat="-",n=2,expand=False).str[0].str.rjust(5, "0")
print(df1_pd.head(5))
end = perf_counter()
print(f"Spent {round(end-start,3)}s.")

        day        country postal_code  total_sales  orders postal_code_2
0  7/1/2022  United States       84115     6.384645       1         84115
1  7/1/2022  United States  91326-1749     2.113960       1         91326
2  7/1/2022  United States       93103    20.224295       1         93103
3  7/1/2022  United States        2465    94.731130       1         02465
4  7/1/2022  United States       80108    52.652385       1         80108
Spent 0.114s.


# Filtering 

1. country = "United States"

In [12]:
start = perf_counter()
df1_pl = df1_pl.filter(pl.col("country") == "United States")
end = perf_counter()
print(f"Spent {round(end-start,3)}s.")

Spent 0.004s.


In [13]:
start = perf_counter()
df1_pd = df1_pd[df1_pd["country"] == "United States"]
end = perf_counter()
print(f"Spent {round(end-start,3)}s.")

Spent 0.016s.


# Joining 

In [14]:
dim_pl = pl.read_csv("./data/geo_zip_dim.csv", dtypes={"postal_code":str})
dim_pd = pd.read_csv("./data/geo_zip_dim.csv", dtype={"postal_code":str})

In [15]:
dim_pl.limit(5)

postal_code,dma_code,dma_name
str,i64,str
"""11005""",501,"""New York"""
"""11010""",501,"""New York"""
"""11020""",501,"""New York"""
"""11021""",501,"""New York"""
"""11023""",501,"""New York"""


In [16]:
start = perf_counter()
df1_pl = df1_pl.join(dim_pl, left_on="postal_code_2", right_on="postal_code", how="left")
end = perf_counter()
print(f"Spent {round(end-start,3)}s.")
print(df1_pl.limit(5))


Spent 0.005s.
shape: (5, 8)
┌──────────┬─────────┬───────────┬────────────────┬────────┬────────────┬──────────┬───────────────┐
│ day      ┆ country ┆ postal_co ┆ total_sales    ┆ orders ┆ postal_cod ┆ dma_code ┆ dma_name      │
│ ---      ┆ ---     ┆ de        ┆ ---            ┆ ---    ┆ e_2        ┆ ---      ┆ ---           │
│ str      ┆ str     ┆ ---       ┆ f64            ┆ i64    ┆ ---        ┆ i64      ┆ str           │
│          ┆         ┆ str       ┆                ┆        ┆ str        ┆          ┆               │
╞══════════╪═════════╪═══════════╪════════════════╪════════╪════════════╪══════════╪═══════════════╡
│ 7/1/2022 ┆ United  ┆ 84115     ┆ 6.384645       ┆ 1      ┆ 84115      ┆ 770      ┆ Salt Lake     │
│          ┆ States  ┆           ┆                ┆        ┆            ┆          ┆ City          │
│ 7/1/2022 ┆ United  ┆ 91326-174 ┆ 2.11396        ┆ 1      ┆ 91326      ┆ 803      ┆ Los Angeles   │
│          ┆ States  ┆ 9         ┆                ┆        ┆   

In [17]:
start = perf_counter()
df1_pd = df1_pd.merge(dim_pd, left_on="postal_code_2", right_on="postal_code", how="left")
end = perf_counter() 
print(f"Spent {round(end-start,3)}s.")
print(df1_pd.head(5))

Spent 0.037s.
        day        country postal_code_x  total_sales  orders postal_code_2  \
0  7/1/2022  United States         84115     6.384645       1         84115   
1  7/1/2022  United States    91326-1749     2.113960       1         91326   
2  7/1/2022  United States         93103    20.224295       1         93103   
3  7/1/2022  United States          2465    94.731130       1         02465   
4  7/1/2022  United States         80108    52.652385       1         80108   

  postal_code_y  dma_code                    dma_name  
0         84115     770.0              Salt Lake City  
1         91326     803.0                 Los Angeles  
2         93103     855.0  Santabarbra-Sanmar-Sanluob  
3         02465     506.0         Boston (Manchester)  
4         80108     751.0                      Denver  


# Grouping 

In [19]:
print(len(df1_pl))
start = perf_counter()
df1_pl_agg = df1_pl.groupby(["dma_code", "dma_name"])\
            .agg([
                pl.col("postal_code_2").n_unique().alias("ZipCodesInDMA"),
                pl.col("total_sales").sum()
            ]).sort("dma_name", nulls_last=True)
end = perf_counter() 
print(f"Spent {round(end-start,3)}s.")
print(df1_pl_agg.limit(5))

84288
Spent 0.003s.
shape: (5, 4)
┌──────────┬─────────────────────────┬───────────────┬─────────────┐
│ dma_code ┆ dma_name                ┆ ZipCodesInDMA ┆ total_sales │
│ ---      ┆ ---                     ┆ ---           ┆ ---         │
│ i64      ┆ str                     ┆ u32           ┆ f64         │
╞══════════╪═════════════════════════╪═══════════════╪═════════════╡
│ 662      ┆ Abilene-Sweetwater      ┆ 8             ┆ 643.593381  │
│ 525      ┆ Albany, GA              ┆ 8             ┆ 868.506348  │
│ 532      ┆ Albany-Schenectady-Troy ┆ 69            ┆ 8595.964934 │
│ 790      ┆ Albuquerque-Santa Fe    ┆ 34            ┆ 7881.710945 │
│ 644      ┆ Alexandria, LA          ┆ 5             ┆ 331.027846  │
└──────────┴─────────────────────────┴───────────────┴─────────────┘


In [24]:
print(len(df1_pd))
start = perf_counter()
df1_pd_agg = df1_pd.groupby(["dma_code", "dma_name"], as_index=False)\
            .agg({
                "postal_code_2" : "nunique",
                "total_sales" : "sum"
            }).sort_values("dma_name")
end = perf_counter() 
print(f"Spent {round(end-start,3)}s.")
print(df1_pd_agg.head(5))

84288
Spent 0.031s.
     dma_code                 dma_name  postal_code_2  total_sales
129     662.0       Abilene-Sweetwater              8   643.593381
25      525.0               Albany, GA              8   868.506348
32      532.0  Albany-Schenectady-Troy             69  8595.964934
184     790.0     Albuquerque-Santa Fe             34  7881.710945
117     644.0           Alexandria, LA              5   331.027846


In [21]:
%%timeit 

_ = df1_pl.groupby(["dma_code", "dma_name"])\
        .agg([
            pl.col("postal_code_2").n_unique().alias("ZipCodesInDMA"),
            pl.col("total_sales").sum()
        ])

1.74 ms ± 28.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
%%timeit 
_ = df1_pd.groupby(["dma_code", "dma_name"], sort=False, as_index=False)\
            .agg({
                "postal_code_2" : "nunique",
                "total_sales" : "sum"
            })

20.5 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
